# EEC 174AY Lab B2 

In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.text_cell_render p{
    font-size: 130%;
    line-height: 125%;
}
</style>
""")

## Outline

This lab will build your skills in utilizing LSTM networks so that you can apply deep learning to time series information

1. you will code an LSTM network and apply it to a pre-built codebase. Your focus will be on the ML coding
2. You will utilize a partially built code base and then finish it to detect ARDS in waveform data.

## LSTM Network

LSTM is a network that is able to utilize time series information and learn long term patterns to make more accurate predictions than a normal neural network would be able to. We show the general network architecture as an instruction for what you will need to code.

<img src="The_LSTM_cell.png" width=55% height=auto\>

You will be applying LSTM to the task of patient ventilator asynchrony (PVA) detection. We have supplied a bit of the code you will need. Your jobs will be the following:

1. Code the `find_scaling_coefs`, `scale_breath`, and `pad_or_cut_breath` methods in the `PVADataset` class in `dataset.py`.
2. Code a simple 1 layer LSTM network based on network schematics given above. You are welcome to use other resource for assistance as well.
3. Run your LSTM model on PVA detection. How well does your model perform compared to your original Random Forest classifier? Why are you getting these results?
4. Code a more complex 3 layer LSTM network. Do additional layers improve results? Why/Why not?

For the math required we would advise you follow the [PyTorch LSTM mathematics](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) 

In [5]:
from torch import nn
from torch.nn import functional as F


class LSTMNetwork(nn.Module):
    def __init__(self):
        super(LSTMNetwork, self).__init__()
        # Feel free to modify this
        lstm_hidden_units = 32
        # First layer is defined for you. Only have 2 input features (flow, pressure)
        self.ii = nn.Linear(2, lstm_hidden_units)
        # XXX TODO
        
        # Final layer is defined for you too. Have 3 potential output classes (normal, bsa, dta)
        self.final_classification = nn.Linear(lstm_hidden_units, 3)
        
    def forward(self, x):
        # XXX code this up
        pass
    

In [4]:
from pathlib import Path

import pandas as pd
import torch
from torch import nn
from torch.autograd import Variable
from torch.optim import SGD
from torch.utils.data import DataLoader

from dataset import PVADataset

model = LSTMNetwork().cuda()
# You should modify the learning rate as suits the problem
optimizer = SGD(model.parameters(), lr=0.01)
bce = nn.BCEWithLogitsLoss()
batch_size = 16


def get_dataset(path, name):
    saved_set = Path(path)
    # Make sure we save previously processed data. This speeds up future processes.
    if saved_set.exists():
        dataset = pd.read_pickle(saved_set.resolve())
    else:
        # use a sequence length of 224 inputs. If you want to shorten this feel free.
        dataset = PVADataset(name, 224)
        dataset.process_dataset()
        pd.to_pickle(dataset, saved_set.resolve())
    return dataset
        
        
def get_all_datasets():
    training_set = get_dataset('pva_training_set.pkl', 'train')
    validation_set = get_dataset('pva_validation_set.pkl', 'val')
    testing_set = get_dataset('pva_testing_set.pkl', 'test')
    return training_set, validation_set, testing_set



def perform_training_epoch(train_loader):
    with torch.enable_grad():
        for x, y in train_loader:
            x = Variable(x.float()).cuda()
            y = Variable(y.float()).cuda()
            output = model(x)
            loss = bce(output, y)
            loss.backward()
            optimizer.step()
            # Feel free to add diagnostic information like printing loss.
            # I will let you handle this.

                
def perform_inferencing(loader):
    with torch.no_grad():
        for x, y in loader:
            output = model(x.float()).cuda()
            # you must code lines that evaluate performance of the model
            # you can use a number of different metrics including loss,
            # accuracy, sensitivity, specificity, precision, and f1 score.
            #
            # For assignment reporting we will require that you use f1 score.
            

training_set, validation_set, testing_set = get_all_datasets()
# XXX make sure val and testing share same coefficients as training set!!

train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testing_set, batch_size=batch_size, shuffle=False)
# You can write up the rest of the code here. We have already given you most of
# what you need to run the module yourself.

Exception: you need to code me before things will run

## ARDS Detection

Regardless of whether you were successful on your last assignment, the design was to show you the internal mechanism about how LSTM works.

In this assignment you will utilize a dataset of ventilation data taken from 50 subjects. 25 subjects have ARDS, 25 subjects do not have ARDS. Your job is to extract waveform data, and utilize it to perform inferencing on whether the patient has ARDS or not.

1. Use basic CNN architecture to perform classification on whether patient has ARDS or not
2. Add LSTM to CNN architecture, do results improve? if not why? In this assignment you should use the [PyTorch LSTM layer.](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

### Data

The data that we use here is ventilation data but it is structured a bit differently than the PVA dataset. Primarily, the data is structured in continuous breath sequences instead of single breaths. Here is an example.

<img src=ards-data.png width=50% height=auto\>

This has a few advantages: 

1. We don't need padding anymore
2. It improves performance of our model

We stack 20 of these breaths together into a tensor that is in shape `(20, 1, 224)`. This allows us to analyze sequential breaths with an LSTM if we desire. 

In [4]:
from pathlib import Path

from torch.utils.data import DataLoader

from dataset import ARDSDataset

batch_size = 32


def get_dataset(path, name):
    saved_set = Path(path)
    # Make sure we save previously processed data. This speeds up future processes.
    if saved_set.exists():
        dataset = ARDSDataset.from_pickle(saved_set.resolve())
    else:
        dataset = ARDSDataset(224, name, to_pickle=saved_set.resolve())
    return dataset
        
        
def get_all_datasets():
    training_set = get_dataset('ards_training_set.pkl', 'train')
    validation_set = get_dataset('ards_validation_set.pkl', 'val')
    testing_set = get_dataset('ards_testing_set.pkl', 'test')
    return training_set, validation_set, testing_set


training_set, validation_set, testing_set = get_all_datasets()
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testing_set, batch_size=batch_size, shuffle=False)

In [5]:
from torch import nn
from torch.optim import SGD

from cnn_lstm_net import CNNLSTMNetwork
from densenet import densenet18


# You are welcome to evaluate other CNN backbones
cnn = densenet18()

# feel free to modify these parameters
lstm_hidden_units = 128
lstm_layers = 1

# 0 means there is 0% probability of dropout happening
lstm_dropout = 0

model = CNNLSTMNetwork(cnn, lstm_hidden_units, lstm_layers, lstm_dropout)

# We highly recommend using SGD for this problem
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
bce = nn.BCEWithLogitsLoss()